In [37]:
import opensim as osim
import json
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Optional, Dict
from math import pi, atan2, sqrt, acos, sin, cos

# Vesrion 2 From Original Code

## Loading the model and getting information of the model 

In [38]:
def load_and_analyze_model(model_path):
    """
    Load OpenSim model and extract all important information
    
    Args:
        model_path (str): Path to .osim model file
    
    Returns:
        dict: Model information
    """
    try:
        # Load the model
        print(f"Loading model: {model_path}")
        model = osim.Model(model_path)
        
        # Initialize the system (important step!)
        state = model.initSystem()
        
        print(f"✓ Model loaded successfully: {model.getName()}")
        print(f"✓ Model description: {model.getDescription()}")
        print("=" * 50)
        
        # Get model information
        model_info = {
            'name': model.getName(),
            'description': model.getDescription(),
            'coordinates': [],
            'bodies': [],
            'joints': [],
            'muscles': []
        }
        
        # 1. COORDINATES (joint angles/positions)
        print("\n COORDINATES (Joint Angles):")
        coord_set = model.getCoordinateSet()
        print(f"Total coordinates: {coord_set.getSize()}")
        
        for i in range(coord_set.getSize()):
            coord = coord_set.get(i)
            coord_info = {
                'name': coord.getName(),
                'motion_type': coord.getMotionType().name if hasattr(coord.getMotionType(), 'name') else str(coord.getMotionType()),
                'default_value': coord.getDefaultValue(),
                'range_min': coord.getRangeMin(),
                'range_max': coord.getRangeMax(),
                'locked': coord.getDefaultLocked()
            }
            model_info['coordinates'].append(coord_info)
            
            print(f"  {i+1}. {coord.getName()}")
            print(f"     Type: {coord_info['motion_type']}")
            print(f"     Range: [{coord.getRangeMin():.3f}, {coord.getRangeMax():.3f}]")
            print(f"     Default: {coord.getDefaultValue():.3f}")
            print(f"     Locked: {coord.getDefaultLocked()}")
        
        # 2. BODIES (rigid body segments)
        print(f"\n BODIES (Rigid Segments):")
        body_set = model.getBodySet()
        print(f"Total bodies: {body_set.getSize()}")
        
        for i in range(body_set.getSize()):
            body = body_set.get(i)
            body_info = {
                'name': body.getName(),
                'mass': body.getMass()
            }
            model_info['bodies'].append(body_info)
            
            print(f"  {i+1}. {body.getName()}")
            print(f"     Mass: {body.getMass():.3f} kg")
        
        # 3. JOINTS (connections between bodies)
        print(f"\n🔗 JOINTS (Connections):")
        joint_set = model.getJointSet()
        print(f"Total joints: {joint_set.getSize()}")
        
        for i in range(joint_set.getSize()):
            joint = joint_set.get(i)
            joint_info = {
                'name': joint.getName(),
                'type': joint.getConcreteClassName(),
                'parent_body': joint.getParentFrame().getName(),
                'child_body': joint.getChildFrame().getName()
            }
            model_info['joints'].append(joint_info)
            
            print(f"  {i+1}. {joint.getName()}")
            print(f"     Type: {joint.getConcreteClassName()}")
            print(f"     Connects: {joint.getParentFrame().getName()} → {joint.getChildFrame().getName()}")
        
        # 4. MUSCLES (if any)
        print(f"\n MUSCLES:")
        muscle_set = model.getMuscles()
        print(f"Total muscles: {muscle_set.getSize()}")
        
        if muscle_set.getSize() > 0:
            for i in range(min(5, muscle_set.getSize())):  # Show first 5 only
                muscle = muscle_set.get(i)
                muscle_info = {
                    'name': muscle.getName(),
                    'max_force': muscle.getMaxIsometricForce()
                }
                model_info['muscles'].append(muscle_info)
                
                print(f"  {i+1}. {muscle.getName()}")
                print(f"     Max Force: {muscle.getMaxIsometricForce():.1f} N")
            
            if muscle_set.getSize() > 5:
                print(f"  ... and {muscle_set.getSize() - 5} more muscles")
        else:
            print("No muscles in this model")
        
        return model_info, model, state
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None
    

## Creating two link arm model to solve the kinematics of the model and finding angles

In [39]:
class Arm2D:
    """"Simple 2D arm model with two links"""
    def __init__(self, link1_length, link2_length):
        """
        Initialize 2-link arm
        
        Args:
            Length1 (float): Upper arm length (meters)
            Length2 (float): Forearm length (meters)
        """
        self.link1_length = link1_length # upper arm length
        self.link2_length = link2_length #  forearm length

        self.max_reach = link1_length + link2_length # maximum reach of the arm
        if self.max_reach < 0:
            self.max_reach = 0
        self.min_reach = abs(link1_length - link2_length) # minimum reach of the arm
        if self.min_reach < 0:
            self.min_reach = 0

        # Print initialization details
        print(f"Arm initialized with lengths: {link1_length}m (link1), {link2_length}m (link2)")
        print(f"Max reach: {self.max_reach}m, Min reach: {self.min_reach}m")

    def inverse_kinematics(self, target_x, target_y):
        """
        Solve inverse kinematics for 2-link arm
        
        Args:
            target_x, target_y (float): Target position
            
        Returns:
            tuple: (shoulder_angle, elbow_angle) in radians
                   Returns None if target unreachable
        """
        d = sqrt(target_x**2 + target_y**2)
        if d > (self.max_reach):
            print("Target too far: {r:.3f} > {self.max_reach:.3f}")
            return None
        
        if d < (self.min_reach):
            print("Target too close: {r:.3f} < {self.min_reach:.3f}")
            return None
        # Calculate angles using cosine law
        cos_elbow = (d**2 - self.link1_length**2 - self.link2_length**2) / (2 * self.link1_length * self.link2_length)
        
        # To acoount for numerical errors, we clamp the value
        cos_elbow = max(-1.0, min(1.0, cos_elbow))
        elbow_angle = acos(cos_elbow)
        
        # Shoulder angle
        alpha = atan2(target_y, target_x) # Angel to the target point
        
        # This is for an elbow down configuration I dont know if it the best when it is more complex, and dont know 
        # if I need to add a check for elbow up configuration,
        # For reaching movements, elbow down is usually preferred
        if d > 0:
            beta = acos((self.link1_length**2 + d**2 - self.link2_length**2) / (2 * self.link1_length * d))
            shoulder_angle = alpha - beta
        else:
            shoulder_angle = 0
        
        
        return shoulder_angle, elbow_angle
    
    def forward_kinematics(self, shoulder_angle, elbow_angle):
        """
        Calculate end-effector position from joint angles
        
        Args:
            shoulder_angle, elbow_angle (float): Joint angles in radians
            
        Returns:
            tuple: (x, y) position of end-effector
        """
        # Calculate end-effector position using forward kinematics
        end_x = self.link1_length * cos(shoulder_angle) + self.link2_length * cos(shoulder_angle + elbow_angle)
        end_y = self.link1_length * sin(shoulder_angle) + self.link2_length * sin(shoulder_angle + elbow_angle)
        return end_x, end_y
    
    def plot_arm_config(self, shoulder_angle, elbow_angle, target=None):
        """
        Plot the arm configuration
        
        Args:
            shoulder_angle, elbow_angle (float): Joint angles in radians
        """
        
        shoulder_pos = (0, 0)  # Shoulder position at origin
        # Calculate positions
        elbow_x = self.link1_length * cos(shoulder_angle)
        elbow_y = self.link1_length * sin(shoulder_angle)
        elbow_pos = (elbow_x, elbow_y)

        end_x = elbow_x + self.link2_length * cos(shoulder_angle + elbow_angle)
        end_y = elbow_y + self.link2_length * sin(shoulder_angle + elbow_angle)
        end_pos = (end_x, end_y)
        
        # Plot the arm
        plt.figure(figsize=(8, 6))

        # Draw arm links
        plt.plot([shoulder_pos[0], elbow_pos[0]], [shoulder_pos[1], elbow_pos[1]], 'b-', linewidth=8, label='Upper Arm', alpha=0.7)
        plt.plot([elbow_pos[0], end_pos[0]], [elbow_pos[1], end_pos[1]], 'r-', linewidth=8, label='Forearm', alpha=0.7)
        # plt.plot(x2, y2, 'go', markersize=10, label='End Effector')
        
        # Draw joints
        plt.plot(shoulder_pos[0], shoulder_pos[1], 'ko', markersize=10, label='Shoulder')
        plt.plot(elbow_pos[0], elbow_pos[1], 'go', markersize=10, label='Elbow')
        plt.plot(end_pos[0], end_pos[1], 'ro', markersize=10, label='End Effector')

        # Draw target if provided
        if target:
            plt.plot(target[0], target[1], 'mx', markersize=12, label='Target', alpha=0.7)
            plt.text(target[0], target[1], f'Target ({target[0]:.2f}, {target[1]:.2f})', fontsize=10, ha='right')

        # Draw workspace circle
        circle = plt.Circle((0, 0), self.max_reach, color='gray', fill=False, linestyle='--', label='Max Reach', alpha=0.3)
        plt.gca().add_patch(circle)


        plt.xlim(-self.max_reach - 0.1, self.max_reach + 0.1)
        plt.ylim(-self.max_reach - 0.1, self.max_reach + 0.1)
        plt.axhline(0, color='k', linewidth=0.5)
        plt.axvline(0, color='k', linewidth=0.5)
        
        # Setting limits
        limit = self.max_reach * 1.1
        plt.xlim(-limit, limit)
        plt.ylim(-limit, limit)

        # Formatting
        plt.axis('equal')
        plt.grid(True, alpha=0.5)
        plt.legend()
        plt.xlabel('X Position (m)')
        plt.ylabel('Y Position (m)')
        plt.title(f'2D Arm Configuration\nShoulder: {np.rad2deg(shoulder_angle):.2f}°, Elbow: {np.rad2deg(elbow_angle):.2f}°')

        return end_pos

## Trajectory planning, generating and path

In [40]:
class TrajectoryGenerator:
    """ Generate smooth natuaral trajectories for reachcing movements """
    def __init__(self):
        self.trajectory_types = {
            'linear': self.generate_linear_trajectory,
            'minimum_jerk': self.generate_minimum_jerk_trajectory,
            'bell shape': self.generate_bell_trajectory,
            'polynomial': self.generate_polynomial_trajectory
        }
    
    def generate_linear_trajectory(self, start, end, duration, num_points):
        """simple linear trajectory for comparison"""
        t = np.linspace(0, duration, num_points)
        trajectory = np.zeros((num_points,3))

        for i in range(3):
            trajectory[:, i] = np.linspace(start[i], end[i], num_points)

        return trajectory, t
    
    def generate_minimum_jerk_trajectory(self, start, end, duration, num_points):
        """
        Generate a minimum jerk trajectory - most natural for human reaching
        We will use a 5th order polynomial to generate the trajectory and smooth accerleration
        """
        t = np.linspace(0, duration, num_points)
        trajectory = np.zeros((num_points, 3))

        for i in range(3):
            s = start[i] # start position
            e = end[i]   # end position

            # tau normalized time
            tau = t / duration

            # 5th order polynomial  minimum jerk
            trajectory[:, i] = s + (e - s) * (10 * tau**3 - 15 * tau**4 + 6 * tau**5)

        return trajectory, t
    
    def generate_bell_trajectory(self, start, end, duration, num_points):
        """
        Generate a bell-shaped trajectory - smooth and natural
        For now I am gonna use a sin function to generate a bell-shaped trajectory
        I have to change it to a gussian function or something similar
        """
        t = np.linspace(0, duration, num_points)
        trajectory = np.zeros((num_points, 3))

        for i in range(3):
            s = start[i] # start position
            e = end[i] # end position

            # tau normalized time
            tau = t / duration
            # Bell-shaped trajectory
            trajectory[:, i] = s + (e - s) * (tau - np.sin(2 * pi * tau) / (2 * pi))

        return trajectory, t
    
    def generate_polynomial_trajectory(self, start, end, duration, num_points,order=3):
        """
        Generic polynomial trajectory
        Higher order = smoother but more oscillatory
        """
        t = np.linspace(0, duration, num_points)
        trajectory = np.zeros((num_points, 3))

        for i in range(3):
            s = start[i] # start position
            e = end[i]   # end position
            # tau normalized time
            tau = t / duration

            if order == 3:
                # Cubic polynomial
                trajectory[:, i] = s + (e - s) * (3 * tau**2 - 2 * tau**3)
            elif order == 5:
                # 5th order polynomial
                trajectory[:, i] = s + (e - s) * (10 * tau**3 - 15 * tau**4 + 6 * tau**5)
            else:
                # Linear fallback
                trajectory[:, i] = s + (e - s) * tau

        return trajectory, t
    
    def via_point_trajectory(self,waypoints, duration, num_points, trajectory_type='minimum_jerk'):
        """
        Generate trajectory through multiple waypoints
        
        Args:
            waypoints (list): List of (x, y, z) positions
            duration (float): Total movement time
            num_points (int): Total trajectory points
            trajectory_type (str): Type of trajectory between points
        """
        if len(waypoints) < 2:
            raise ValueError("At least two waypoints (start and end) are required.")
        
        num_segments = len(waypoints) - 1
        segment_duration = duration / num_segments
        points_per_segment = num_points // num_segments

        full_trajectory = []
        full_time = []
        current_time = 0

        for i in range(num_segments):
            start_point = waypoints[i]
            end_point = waypoints[i + 1]

            # Generate trajectory for this segment
            segemnt_trajectory, segment_time = self.trajectory_types[trajectory_type](start_point, 
                                                                                      end_point, segment_duration, points_per_segment)
            
            # add to full trajectory
            if i == 0:
                full_trajectory.extend(segemnt_trajectory)
                full_time.extend(segment_time)
            else:
                full_trajectory.extend(segemnt_trajectory[1:])  # Skip first point to avoid duplication
                full_time.extend(segment_time[1:])  # Skip first time point to avoid duplication
            # Update current time
            current_time += segment_duration
            

        
        return np.array(full_trajectory), np.array(full_time)


def calculate_motion_profiles(trajectory, time):
    """    Calculate velocity and acceleration profiles from trajectory
    Args:
        trajectory (np.ndarray): Trajectory points (N, 3)
        time (np.array): Time array
    Returns:
        tuple: (velocity, acceleration, velocity_magnitude, acceleration_magnitude)
    """

    dt = time[1] - time[0]  if len(time) > 1 else 0.01 

    # Calculate velocity first derivative
    velocity = np.gradient(trajectory, dt, axis=0)

    # Calculate acceleration second derivative
    acceleration = np.gradient(velocity, dt, axis=0)

    # calculate the magnitude of velocity and acceleration  
    velocity_magnitude = np.linalg.norm(velocity, axis=1)
    acceleration_magnitude = np.linalg.norm(acceleration, axis=1)

    return velocity, acceleration, velocity_magnitude, acceleration_magnitude

def trajectory_to_joint_motion(trajectory, time, arm):
    """
    Convert smooth trajectory to smooth joint angles
    This is the KEY CONNECTION between Day 2 IK and Day 3 trajectories!
    
    Args:
        trajectory (np.array): Smooth trajectory points (n_points x 3)
        time (np.array): Time array
        arm (Arm2D): The IK solver from Day 2
        
    Returns:
        tuple: (shoulder_angles, elbow_angles, valid_times, reachable_positions)
    """
    print("🔗 Connecting trajectory to IK solver...")
    print(f"   Processing {len(trajectory)} trajectory points...")
    
    # Storage for results
    shoulder_angles = []
    elbow_angles = []
    valid_times = []
    reachable_positions = []
    unreachable_count = 0
    
    # Process each point in trajectory
    for i, pos in enumerate(trajectory):
        # Extract x, y from trajectory point (ignore z for 2D arm)
        target_x, target_y = pos[0], pos[1]
        
        # Solve IK for this position (Day 2 function!)
        ik_result = arm.inverse_kinematics(target_x, target_y)
        
        if ik_result is not None:
            shoulder, elbow = ik_result
            
            # Store the joint angles and corresponding data
            shoulder_angles.append(shoulder)
            elbow_angles.append(elbow)
            valid_times.append(time[i])
            reachable_positions.append(pos)
        else:
            unreachable_count += 1
            if unreachable_count <= 3:  # Only print first few
                print(f"   ⚠️  Point {i} unreachable: ({target_x:.3f}, {target_y:.3f})")
    
    if unreachable_count > 3:
        print(f"   ⚠️  ... and {unreachable_count - 3} more unreachable points")
    
    success_rate = len(shoulder_angles) / len(trajectory) * 100
    print(f"   ✅ Success: {len(shoulder_angles)}/{len(trajectory)} points ({success_rate:.1f}%)")
    
    return shoulder_angles, elbow_angles, valid_times, reachable_positions

def demonstrate_trajectory_ik_connection():
    """
    Demonstrate the connection between smooth trajectories and IK solver
    This is the missing piece you asked about!
    """
    print("\n🎯 DEMONSTRATING TRAJECTORY ↔ IK CONNECTION")
    print("=" * 50)
    
    # Create arm and trajectory generator
    arm = Arm2D(0.3, 0.25)
    generator = TrajectoryGenerator()
    
    # Generate different trajectory types
    start_pos = [0.2, 0.35, 0.0]
    end_pos = [0.4, 0.15, 0.0]
    duration = 2.0
    num_points = 50
    
    trajectory_types = ['linear', 'minimum_jerk']
    results = {}
    
    for traj_type in trajectory_types:
        print(f"\n📈 Processing {traj_type.upper()} trajectory...")
        
        # Step 1: Generate trajectory (Day 3)
        trajectory, time = generator.trajectory_types[traj_type](
            start_pos, end_pos, duration, num_points
        )
        
        # Step 2: Convert to joint angles (Day 2 IK)
        shoulder_angles, elbow_angles, valid_times, reachable_pos = trajectory_to_joint_motion(
            trajectory, time, arm
        )
        
        results[traj_type] = {
            'trajectory': trajectory,
            'time': time,
            'shoulder_angles': shoulder_angles,
            'elbow_angles': elbow_angles,
            'valid_times': valid_times,
            'reachable_positions': reachable_pos
        }
    
    # Plot the connection
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    colors = {'linear': 'red', 'minimum_jerk': 'blue'}
    
    # Plot 1: Trajectory paths (what Day 3 generates)
    ax = axes[0, 0]
    for traj_type, data in results.items():
        traj = data['trajectory']
        ax.plot(traj[:, 0], traj[:, 1], color=colors[traj_type], 
                linewidth=2, label=f'{traj_type} trajectory')
    
    ax.plot(start_pos[0], start_pos[1], 'go', markersize=8, label='Start')
    ax.plot(end_pos[0], end_pos[1], 'ro', markersize=8, label='End')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title('INPUT: Smooth Trajectories (Day 3)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    
    # Plot 2: Joint angles (what Day 2 IK produces)
    ax = axes[0, 1]
    for traj_type, data in results.items():
        times = data['valid_times']
        shoulders = np.rad2deg(data['shoulder_angles'])
        ax.plot(times, shoulders, color=colors[traj_type], 
                linewidth=2, label=f'{traj_type} shoulder')
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Shoulder Angle (degrees)')
    ax.set_title('OUTPUT: Smooth Joint Angles (Day 2 IK)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Compare smoothness of joint angles
    ax = axes[1, 0]
    for traj_type, data in results.items():
        times = data['valid_times']
        elbows = np.rad2deg(data['elbow_angles'])
        ax.plot(times, elbows, color=colors[traj_type], 
                linewidth=2, label=f'{traj_type} elbow')
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Elbow Angle (degrees)')
    ax.set_title('Elbow Angle Smoothness Comparison')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Joint velocities (derivative of angles)
    ax = axes[1, 1]
    for traj_type, data in results.items():
        times = data['valid_times']
        shoulders = data['shoulder_angles']
        
        if len(times) > 1:
            dt = times[1] - times[0]
            shoulder_vel = np.gradient(shoulders, dt)
            ax.plot(times, np.rad2deg(shoulder_vel), color=colors[traj_type], 
                    linewidth=2, label=f'{traj_type} shoulder vel')
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angular Velocity (deg/s)')
    ax.set_title('Joint Velocity Smoothness')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show the key insight
    print("\n🔍 KEY INSIGHT:")
    print("   Smooth trajectory → Smooth joint angles → Natural movement!")
    
    for traj_type, data in results.items():
        if len(data['shoulder_angles']) > 1:
            # Calculate smoothness metric
            dt = data['valid_times'][1] - data['valid_times'][0]
            shoulder_vel = np.gradient(data['shoulder_angles'], dt)
            smoothness = np.std(shoulder_vel)  # Lower = smoother
            
            print(f"   {traj_type.capitalize()}: joint velocity std = {smoothness:.4f}")
    
    return results


## Integration with OpenSim Maybe work maybe not 


In [41]:
class OpenSimReachingSystem:
    """
    Compelete system for generating the smooth trajectory reaching movemenets in OPENSIM.
    Integrates model loading, inverse kinematics and trajectory generation.
    """

    def __init__(self, model_path: str):
        """
        Initialize the reaching system with an OpenSim model.
        
        Args:
            model_path: Path to the .osim model file
        """
        self.model_path = model_path
        self.model = None
        self.state = None
        self.coordinate_info = {}
        self.arm_geometry = {}

        # Initialize components
        self.trajectory_generator = TrajectoryGenerator()
        self.arm2D = Arm2D(0.3, 0.25)  # Example arm lengths, can be adjusted

        # Load the model and initialize the model
        self._load_model()
        self.extract_arm_info()

    def _load_model(self):
        """Load the OpenSim model from with the model_analyze_function"""
        print(f"Loading model from {self.model_path}")

        try:
            # Using the model loader
            self.model_info, self.model, self.state = load_and_analyze_model(self.model_path)
            
            if self.model is None:
                raise Exception(f"Failed to load model from {self.model_path}")
            
            print("Model loaded successfully!")
            print(f"   - Name: {self.model_info['name']}")
            print(f"   - Coordinates: {len(self.model_info['coordinates'])}")
            print(f"   - Bodies: {len(self.model_info['bodies'])}")
            print(f"   - Joints: {len(self.model_info['joints'])}")
            print(f"   - Muscles: {len(self.model_info['muscles'])}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def extract_arm_info(self):
        """Extract arm geometry and coordinate info from the model"""
        print("Extracting arm geometry and coordinate info...")
            
        # Assuming the arm is defined by specific coordinates
        self.coordinate_info = {}
        for coord_data in self.model_info['coordinates']:
            self.coordinate_info[coord_data['name']] = {
                'motion_type': coord_data['motion_type'],
                'range_min': coord_data['range_min'],
                'range_max': coord_data['range_max'],
                'default_value': coord_data['default_value'],
                'locked': coord_data['locked']
            }
            
        # Print avaliable coordinates
        print(f"Available coordinates ({len(self.coordinate_info)}):")
        for name, info in self.coordinate_info.items():
            locked_str = "(LOCKED)" if info['locked'] else ""
            print(f"  - {name}: [{info['range_min']:.2f} to {info['range_max']:.2f} rad{locked_str}]")
            print(f"    Motion type: {info['motion_type']}, Default: {info['default_value']:.3f}")

        # identify arm-related coordinates
        self.arm_coordinates = self._identify_arm_coordinates()

        # estimate arm geometry 
        self.arm_geometry = self._estimate_arm_geometry()

        print("\n final arm config:")
        if self.arm_coordinates:
            for joint_type, coord_name in self.arm_coordinates.items():
                print(f"  - {joint_type.capitalize()}: {coord_name} (range: [{self.coordinate_info[coord_name]['range_min']:.2f}, {self.coordinate_info[coord_name]['range_max']:.2f}])")
        else:
            print("  No arm coordinates found, please modify the code to set them manually.")
        


            
    def _identify_arm_coordinates(self) -> dict[str, str]:
        """
        dentify which coordinates correspond to shoulder and elbow using enhanced patterns.
        Returns mapping of joint_type -> coordinate_name
        """
        arm_coords = {}
        coord_names = list(self.coordinate_info.keys())

        print(f"\n🔍 Searching for arm coordinates in {len(coord_names)} available coordinates...")

        # Enhanced naming patterns for arm joints
        shoulder_patterns = [
            'shoulder_flexion', 'shoulder_flex', 'shoulder_elv', 'arm_flex',
            'r_shoulder_flex', 'l_shoulder_flex', 'shoulder_add', 'shoulder_abduction',
            'arm_add', 'arm_abduction', 'glenohumeral'
        ]
        elbow_patterns = [
            'elbow_flexion', 'elbow_flex', 'elbow', 'r_elbow_flex', 'l_elbow_flex',
            'radioulnar', 'humeroulnar'
        ]

        # Find shoulder coordinates
        for pattern in shoulder_patterns:
            matches = [name for name in coord_names if pattern in pattern.lower() in pattern.lower()]
            if matches:
                arm_coords['shoulder'] = matches[0]
                print(f"  Found shoulder coordinate: '{matches[0]}' (matched pattern '{pattern}')")     
                break

        # Find elbow coordinates
        for pattern in elbow_patterns:
            matches = [name for name in coord_names if pattern in pattern.lower() in pattern.lower()]
            if matches:
                arm_coords['elbow'] = matches[0]
                print(f"  Found elbow coordinate: '{matches[0]}' (matched pattern '{pattern}')")
                break      

        # look for potential coordinates that aren't locked
        if not arm_coords:
            print("⚠️ No specific arm coordinates found, checking all unlocked coordinates...")
            unlocked_coords = [name for name, info in self.coordinate_info.items() if not info.get['locked', False]]    
            print(f"  Found {len(unlocked_coords)} unlocked coordinates:")

            if len(unlocked_coords) >= 2:
                # Coordinates with reachable joints
                sutiable_coords = []
                for name in unlocked_coords:
                    info = self.coordinate_info[name]
                    range_size = info['range_max'] - info['range_min']
                    if range_size > 0.5 and range_size< 6.0:
                        sutiable_coords.append(name, range_size) 
                        
                    if len(sutiable_coords) >= 2:
                        # sort by range size and take the first two
                        sutiable_coords.sort(key=lambda x: x[1], reverse=True)
                        arm_coords['shoulder'] = sutiable_coords[0][0]
                        arm_coords['elbow'] = sutiable_coords[1][0]
                        print(f"  Selected shoulder: '{arm_coords['shoulder']}' and elbow: '{arm_coords['elbow']}' based on range size.")

        if not arm_coords:
            print("No suitable arm coordinates found, using default arm2D coordinates.")
            print ("Available coordinates for manual selection:")
            for i, name in enumerate(coord_names):
                info = self.coordinate_info[name]
                locked_str = "(LOCKED)" if info['locked'] else ""
                print(f"      {i}: {name} [{info['range_min']:.2f}, {info['range_max']:.2f}]{locked_str}")
                
            print("    Modify the code to manually set arm_coords = {'shoulder': 'coord_name', 'elbow': 'coord_name'}")
            
        return arm_coords
    
    def _estimate_arm_geometry(self):
        """Estimate arm segment lengths from the OpenSim model using Day 1 body analysis."""
        print(f"\n Estimating arm geometry from model bodies...")
        
        # Default arm geometry
        self.arm_geometry = {
            'upper_arm_length': 0.3,  # 30cm upper arm
            'forearm_length': 0.25,   # 25cm forearm
            'shoulder_pos': (0, 0)    # Shoulder position in 2D space
        }
        
        # Try to estimate from model bodies if available
        if hasattr(self, 'model_info') and 'bodies' in self.model_info:
            print(f"   Found {len(self.model_info['bodies'])} bodies in model:")
            
            # Look for arm-related bodies
            arm_bodies = []
            for body_info in self.model_info['bodies']:
                body_name = body_info['name'].lower()
                if any(keyword in body_name for keyword in ['arm', 'humerus', 'radius', 'ulna', 'forearm']):
                    arm_bodies.append(body_info)
                    print(f"     - {body_info['name']}: {body_info['mass']:.3f} kg")
            
            if arm_bodies:
                print(f"    Found {len(arm_bodies)} arm-related bodies")
                # Could implement more sophisticated length estimation here
                # For now, use reasonable defaults based on typical human proportions
                if len(arm_bodies) >= 2:
                    self.arm_geometry['upper_arm_length'] = 0.32  # Slightly longer for realistic models
                    self.arm_geometry['forearm_length'] = 0.27
            else:
                print(f"    No obvious arm bodies found, using default proportions")
        
        print(f" Final arm geometry:")
        print(f"   - Upper arm length: {self.arm_geometry['upper_arm_length']:.2f}m")
        print(f"   - Forearm length: {self.arm_geometry['forearm_length']:.2f}m")
        print(f"   - Total reach: {self.arm_geometry['upper_arm_length'] + self.arm_geometry['forearm_length']:.2f}m")
        
        # Initialize 2D arm solver with these parameters
        self.arm_2d = Arm2D(
            self.arm_geometry['upper_arm_length'],
            self.arm_geometry['forearm_length']
        )
        print(f"   ✅ 2D arm solver initialized")

    def set_arm_pose(self, shoulder_angle:float, elbow_angle:float):
        """
        Set the arm pose in the OpenSim model.
        
        Args:
            shoulder_angle (float): Shoulder angle in radians
            elbow_angle (float): Elbow angle in radians
        """
        if not self.model or not self.state:
            raise RuntimeError("Model not loaded or state not initialized.")
        
        # Set shoulder angle
        if 'shoulder' in self.arm_coordinates:
            coord_name = self.arm_coordinates['shoulder']
            coord = self.model.getCoordinateSet().get(coord_name)
            # Clamp angle to the coordinate range
            angle_clamped = np.clip(shoulder_angle, 
                                          coord.getRangeMin(self.state), 
                                          coord.getRangeMax(self.state))
            coord.setValue(self.state, angle_clamped)
            
        
        # Set elbow angle
        if 'elbow' in self.arm_coordinates:
            coord_name = self.arm_coordinates['elbow']
            coord = self.model.getCoordinateSet().get(coord_name)
            # Clamp angle to the coordinate range
            angle_clamped = np.clip(elbow_angle, 
                                          coord.getRangeMin(self.state), 
                                          coord.getRangeMax(self.state))
            coord.setValue(self.state, angle_clamped)
            
        
        # Update model state
        self.model.realizePosition(self.state)

    def reach_target(self, target_x: float, target_y: float) -> Tuple[bool, Tuple[float, float]]:
        """
        Perform inverse kinematics to reach a target position.
        
        Args:
            target_x, target_y: Target position in meters
            
        Returns:
            (success, (shoulder_angle, elbow_angle))
        """
        if not self.model or not self.state:
            raise RuntimeError("Model not loaded or state not initialized.")
        
        # Use the 2D arm solver to get joint angles
        ik_result = self.arm_2d.inverse_kinematics(target_x, target_y)
        
        if ik_result is None:
            print(f"⚠️  Target ({target_x:.2f}, {target_y:.2f}) unreachable")
            return False, (0.0, 0.0)
        
        shoulder_angle, elbow_angle = ik_result
        
        # Set the arm pose in the OpenSim model
        self.set_arm_pose(shoulder_angle, elbow_angle)
        
        print(f"✅ Reached target ({target_x:.2f}, {target_y:.2f}) with angles: "
              f"Shoulder={np.rad2deg(shoulder_angle):.1f}°, "
              f"Elbow={np.rad2deg(elbow_angle):.1f}°")
        
        return True, (shoulder_angle, elbow_angle)

    def execute_reaching_movement(self, start_pos: Tuple[float, float], 
                                end_pos: Tuple[float, float],
                                duration: float = 2.0,
                                trajectory_type: str = 'minimum_jerk',
                                num_points: int = 50) -> Dict:
        
        """        
        Execute a reaching movement from start to end position using a smooth trajectory.
        Args:
            start_pos: Starting position (x, y) in meters
            end_pos: Ending position (x, y) in meters
            duration: Duration of the movement in seconds
            trajectory_type: Type of trajectory to generate ('linear', 'minimum_jerk', 'bell shape', 'polynomial')
            num_points: Number of points in the trajectory
        Returns:
            Dict: Contains trajectory points, time array, and joint angles if successful
        """
        # generate the smooth trajectory
        trajectory = self.trajectory_generator.trajectory_types[trajectory_type](start_pos, end_pos, duration, num_points)
        
        # Execute movement with the IK solver for each point in the trajectory
        time_points = []
        positions = []
        joint_angles = []
        success_flags = []

        for i, point in enumerate(trajectory):
            t = i * (duration / num_points - 1)
            target_x, target_y = point[0], point[1]
            success, angles = self.reach_target(target_x, target_y)
            time_points.append(t)
            positions.append((target_x, target_y))
            joint_angles.append(angles)
            success_flags.append(success)
        
        # Prepare results
        results = {
            'tajectory_type': trajectory_type,
            'duration': duration,
            'num_points': num_points,
            'time_points': time_points,
            'positions': positions,
            'joint_angles': joint_angles,
            'success_flags': success_flags,
            'success_rate': sum(success_flags) / len(success_flags) * 100
        }

        print(f"✅ Reaching movement executed with {results['success_rate']:.1f}% success rate")
        return results
    
    def create_motion_file(self, movement_results:Dict, filename:str):
        """
        Create an OpenSim .mot file from movement results.
        
        Args:
            movement_results: Results from execute_reaching_movement()
            filename: Output .mot filename
        """
        print(f"\n creating a motion file: {filename}")

        time_points = movement_results['time_points']
        joint_angles = movement_results['joint_angles']

        with open(filename, 'w') as f:
            # Write Header
            f.write(f"name {filename}\n")
            f.write("data columns {}\n".format(len(self.arm_coords) + 1))
            f.write("data rows {}\n" .format(len(time_points)))
            f.write("range {} {}\n" .format(time_points[0], time_points[-1]))
            f.write("endheader\n")

            # column headers
            headers = ['time']
            for joint_type in ['shoulder','elbow']:
                if joint_type in self.arm_coordinates:
                    headers.append(self.arm_coordinates[joint_type])
            f.write("\t".join(headers) + "\n")

            # Data Rows
            for i, t in enumerate(time_points):
                if i < len(joint_angles) and movement_results['success_flags'][i]:
                    shoulder_angle, elbow_angle = joint_angles[i]
                    data_row = [f"{t:.6f}", f"{shoulder_angle:.6f}", f"{elbow_angle:.6f}"]
                    f.write("\t".join(data_row) + "\n")
        
        print(f"✅ Motion file '{filename}' created successfully!")
        print(f"   - {len(time_points)} time points")
        print(f"   - Load this in OpenSim GUI to visualize the movement!")

## Running a demo code

In [42]:
# Demo and testing functions
def demo_reaching_system(model_path:str):
    """
    Demonstration of the complete reaching system.
    Replace 'your_model.osim' with your actual model path.
    """
    
    print("=" * 60)
    print("🚀 DAY 4: OpenSim Integration Demo")
    print("=" * 60)
    
    try:
        # Initialize the reaching system
        reaching_system = OpenSimReachingSystem(model_path)
        
        # Define a reaching movement
        start_position = (0.2, 0.3)   # Start position (20cm right, 30cm up)
        target_position = (0.4, 0.1)  # Target position (40cm right, 10cm up) 
        
        # Execute smooth reaching movement
        results = reaching_system.execute_reaching_movement(
            start_position, 
            target_position,
            duration=3.0,
            trajectory_type='minimum_jerk',
            num_points=100
        )
        
        # Create OpenSim motion file
        motion_filename = f"reaching_movement_{int(time.time())}.mot"
        reaching_system.create_motion_file(results, motion_filename)
        
        # Print movement summary
        print(f"\n📊 Movement Summary:")
        print(f"   - Trajectory type: {results['trajectory_type']}")
        print(f"   - Duration: {results['duration']:.1f}s")  
        print(f"   - Success rate: {results['success_rate']:.1%}")
        print(f"   - Motion file: {motion_filename}")
        
        print(f"\n🎉 SUCCESS! Your OpenSim model can now perform smooth reaching!")
        print(f"📁 Load '{motion_filename}' in OpenSim GUI to see the animation!")
        
    except FileNotFoundError:
        print(f"❌ Model file '{model_path}' not found.")
        print("📝 Please update the model_path variable with your .osim file path.")
        print("💡 You can download sample models from OpenSim.org")
        
    



if __name__ == "__main__":
    print("🎯 Day 4: OpenSim Integration")
    print("Connecting smooth trajectories to OpenSim models!")
    print("-" * 50)
    
    # Run the demo
    model_path = "W:/Models/Models/Arm26/arm26.osim"
    demo_reaching_system(model_path)

🎯 Day 4: OpenSim Integration
Connecting smooth trajectories to OpenSim models!
--------------------------------------------------
🚀 DAY 4: OpenSim Integration Demo
Arm initialized with lengths: 0.3m (link1), 0.25m (link2)
Max reach: 0.55m, Min reach: 0.04999999999999999m
Loading model from W:/Models/Models/Arm26/arm26.osim
Loading model: W:/Models/Models/Arm26/arm26.osim
✓ Model loaded successfully: arm26
✓ Model description: 

 COORDINATES (Joint Angles):
Total coordinates: 2
  1. r_shoulder_elev
     Type: 1
     Range: [-1.571, 3.142]
     Default: 0.000
     Locked: False
  2. r_elbow_flex
     Type: 1
     Range: [0.000, 2.269]
     Default: 0.000
     Locked: False

 BODIES (Rigid Segments):
Total bodies: 2
  1. r_humerus
     Mass: 1.865 kg
  2. r_ulna_radius_hand
     Mass: 1.534 kg

🔗 JOINTS (Connections):
Total joints: 2
  1. r_shoulder
     Type: CustomJoint
     Connects: ground_offset → r_humerus_offset
  2. r_elbow
     Type: CustomJoint
     Connects: r_humerus_offset → 

IndexError: tuple index out of range

# Version 3

In [ ]:
"""
Day 4: OpenSim Integration - Complete Reaching Movement System
============================================================

This combines all previous work:
- Day 1: Model coordinate extraction
- Day 2: 2D inverse kinematics  
- Day 3: Smooth trajectory generation

Creates smooth, human-like reaching movements in OpenSim models.
"""

import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import json
import os
from typing import List, Tuple, Optional, Dict
import time

class OpenSimReachingSystem:
    """
    Complete system for generating smooth reaching movements in OpenSim models.
    Integrates model loading, inverse kinematics, and trajectory generation.
    """
    
    def __init__(self, model_path: str):
        """
        Initialize the reaching system with an OpenSim model.
        
        Args:
            model_path: Path to the .osim model file
        """
        self.model_path = model_path
        self.model = None
        self.state = None
        self.coordinate_info = {}
        self.arm_geometry = {}
        
        # Initialize components from previous days
        self.trajectory_generator = TrajectoryGenerator()
        self.arm_2d = None
        
        # Load and initialize the model
        self._load_model()
        self._extract_arm_info()
        
    def _load_model(self):
        """Load OpenSim model and initialize state using Day 1 comprehensive analysis."""
        print(f"🔍 Loading and analyzing OpenSim model...")
        
        try:
            # Use the comprehensive Day 1 model loader
            self.model_info, self.model, self.state = load_and_analyze_model(self.model_path)
            
            if self.model is None:
                raise Exception("Failed to load model")
                
            print(f"\n✅ Model loaded and analyzed successfully!")
            print(f"📊 Model Summary:")
            print(f"   - Name: {self.model_info['name']}")
            print(f"   - Coordinates: {len(self.model_info['coordinates'])}")
            print(f"   - Bodies: {len(self.model_info['bodies'])}")
            print(f"   - Joints: {len(self.model_info['joints'])}")
            print(f"   - Muscles: {len(self.model_info['muscles'])}")
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            raise
    
    def _extract_arm_info(self):
        """Extract arm-specific coordinate and geometry information using Day 1 analysis."""
        print("\n🦾 Extracting arm information from detailed model analysis...")
        
        # Use the comprehensive coordinate info from Day 1
        self.coordinate_info = {}
        for coord_data in self.model_info['coordinates']:
            self.coordinate_info[coord_data['name']] = {
                'motion_type': coord_data['motion_type'],
                'range_min': coord_data['range_min'],
                'range_max': coord_data['range_max'],
                'default_value': coord_data['default_value'],
                'locked': coord_data['locked']
            }
        
        # Print available coordinates with more detail
        print(f"📋 Available coordinates ({len(self.coordinate_info)}):")
        for name, info in self.coordinate_info.items():
            locked_str = " (LOCKED)" if info['locked'] else ""
            print(f"   - {name}: [{info['range_min']:.2f}, {info['range_max']:.2f}] rad{locked_str}")
            print(f"     Type: {info['motion_type']}, Default: {info['default_value']:.3f}")
        
        # Try to identify arm coordinates with better pattern matching
        self.arm_coords = self._identify_arm_coordinates()
        
        # Estimate arm segment lengths from model bodies
        self._estimate_arm_geometry()
        
        # Show final arm configuration
        print(f"\n🎯 Final arm configuration:")
        if self.arm_coords:
            for joint_type, coord_name in self.arm_coords.items():
                coord_info = self.coordinate_info[coord_name]
                print(f"   - {joint_type.upper()}: {coord_name}")
                print(f"     Range: [{coord_info['range_min']:.2f}, {coord_info['range_max']:.2f}] rad")
        else:
            print("   ⚠️ No arm coordinates identified - manual setup required")
        
    def _identify_arm_coordinates(self) -> Dict[str, str]:
        """
        Identify which coordinates correspond to shoulder and elbow using enhanced patterns.
        Returns mapping of joint_type -> coordinate_name
        """
        arm_coords = {}
        coord_names = list(self.coordinate_info.keys())
        
        print(f"\n🔍 Searching for arm coordinates in {len(coord_names)} available coordinates...")
        
        # Enhanced naming patterns for arm joints
        shoulder_patterns = [
            'shoulder_flexion', 'shoulder_flex', 'shoulder_elv', 'arm_flex',
            'r_shoulder_flex', 'l_shoulder_flex', 'shoulder_add', 'shoulder_abduction',
            'arm_add', 'arm_abduction', 'glenohumeral'
        ]
        elbow_patterns = [
            'elbow_flexion', 'elbow_flex', 'elbow', 'r_elbow_flex', 'l_elbow_flex',
            'radioulnar', 'humeroulnar'
        ]
        
        # Find shoulder coordinate
        print("   Searching for SHOULDER coordinate...")
        for pattern in shoulder_patterns:
            matches = [name for name in coord_names if pattern.lower() in name.lower()]
            if matches:
                arm_coords['shoulder'] = matches[0]
                print(f"   ✅ Found shoulder: '{matches[0]}' (matched pattern: '{pattern}')")
                break
        
        # Find elbow coordinate  
        print("   Searching for ELBOW coordinate...")
        for pattern in elbow_patterns:
            matches = [name for name in coord_names if pattern.lower() in name.lower()]
            if matches:
                arm_coords['elbow'] = matches[0]
                print(f"   ✅ Found elbow: '{matches[0]}' (matched pattern: '{pattern}')")
                break
        
        # Enhanced fallback - look for rotational coordinates that aren't locked
        if not arm_coords:
            print("   ⚠️ Pattern matching failed - trying fallback method...")
            unlocked_coords = [name for name, info in self.coordinate_info.items() 
                             if not info.get('locked', False)]
            print(f"   Found {len(unlocked_coords)} unlocked coordinates: {unlocked_coords}")
            
            if len(unlocked_coords) >= 2:
                # Prefer coordinates with reasonable joint ranges
                suitable_coords = []
                for name in unlocked_coords:
                    info = self.coordinate_info[name]
                    range_size = info['range_max'] - info['range_min']
                    # Look for coordinates with reasonable joint range (0.5 to 6 radians)
                    if 0.5 < range_size < 6.0:
                        suitable_coords.append((name, range_size))
                
                if len(suitable_coords) >= 2:
                    # Sort by range size and take first two
                    suitable_coords.sort(key=lambda x: x[1], reverse=True)
                    arm_coords['shoulder'] = suitable_coords[0][0]
                    arm_coords['elbow'] = suitable_coords[1][0]
                    print(f"   🤖 Auto-selected by range analysis:")
                    print(f"      - shoulder: {arm_coords['shoulder']}")
                    print(f"      - elbow: {arm_coords['elbow']}")
                elif len(unlocked_coords) >= 2:
                    # Last resort - just use first two unlocked
                    arm_coords['shoulder'] = unlocked_coords[0]
                    arm_coords['elbow'] = unlocked_coords[1]
                    print(f"   🤖 Last resort - using first 2 unlocked coordinates")
        
        # Final check and user guidance
        if not arm_coords:
            print("   ❌ Could not identify arm coordinates automatically")
            print("   💡 Available coordinates for manual selection:")
            for i, name in enumerate(coord_names):
                info = self.coordinate_info[name]
                locked_str = " (LOCKED)" if info.get('locked') else ""
                print(f"      {i}: {name} [{info['range_min']:.2f}, {info['range_max']:.2f}]{locked_str}")
            print("   💡 Modify the code to manually set arm_coords = {'shoulder': 'coord_name', 'elbow': 'coord_name'}")
        
        return arm_coords
    
    def _estimate_arm_geometry(self):
        """Estimate arm segment lengths from the OpenSim model using Day 1 body analysis."""
        print(f"\n🔧 Estimating arm geometry from model bodies...")
        
        # EXPANDED arm geometry for greater reach
        self.arm_geometry = {
            'upper_arm_length': 0.35,  # 35cm upper arm (was 30cm)
            'forearm_length': 0.30,    # 30cm forearm (was 25cm)  
            'shoulder_pos': (0, 0)     # Shoulder position in 2D space
        }
        
        # Try to estimate from model bodies if available
        if hasattr(self, 'model_info') and 'bodies' in self.model_info:
            print(f"   Found {len(self.model_info['bodies'])} bodies in model:")
            
            # Look for arm-related bodies
            arm_bodies = []
            for body_info in self.model_info['bodies']:
                body_name = body_info['name'].lower()
                if any(keyword in body_name for keyword in ['arm', 'humerus', 'radius', 'ulna', 'forearm']):
                    arm_bodies.append(body_info)
                    print(f"     - {body_info['name']}: {body_info['mass']:.3f} kg")
            
            if arm_bodies:
                print(f"   ✅ Found {len(arm_bodies)} arm-related bodies")
                # Could implement more sophisticated length estimation here
                # For now, use reasonable defaults based on typical human proportions
                if len(arm_bodies) >= 2:
                    self.arm_geometry['upper_arm_length'] = 0.32  # Slightly longer for realistic models
                    self.arm_geometry['forearm_length'] = 0.27
            else:
                print(f"   ⚠️ No obvious arm bodies found, using default proportions")
        
        print(f"🔧 Final arm geometry:")
        print(f"   - Upper arm length: {self.arm_geometry['upper_arm_length']:.2f}m")
        print(f"   - Forearm length: {self.arm_geometry['forearm_length']:.2f}m")
        print(f"   - Total reach: {self.arm_geometry['upper_arm_length'] + self.arm_geometry['forearm_length']:.2f}m")
        
        # Initialize 2D arm solver with these parameters
        self.arm_2d = Arm2D(
            self.arm_geometry['upper_arm_length'],
            self.arm_geometry['forearm_length']
        )
        print(f"   ✅ 2D arm solver initialized")
    
    def set_arm_pose(self, shoulder_angle: float, elbow_angle: float):
        """
        Set the arm pose in the OpenSim model.
        
        Args:
            shoulder_angle: Shoulder flexion angle in radians
            elbow_angle: Elbow flexion angle in radians
        """
        if 'shoulder' in self.arm_coords:
            coord_name = self.arm_coords['shoulder']
            coord = self.model.getCoordinateSet().get(coord_name)
            
            # Clamp to coordinate limits
            angle_clamped = np.clip(shoulder_angle, 
                                  coord.getRangeMin(), 
                                  coord.getRangeMax())
            coord.setValue(self.state, angle_clamped)
        
        if 'elbow' in self.arm_coords:
            coord_name = self.arm_coords['elbow']
            coord = self.model.getCoordinateSet().get(coord_name)
            
            # Clamp to coordinate limits
            angle_clamped = np.clip(elbow_angle,
                                  coord.getRangeMin(),
                                  coord.getRangeMax())
            coord.setValue(self.state, angle_clamped)
        
        # Realize the state to update positions
        self.model.realizePosition(self.state)
    
    def reach_to_target(self, target_x: float, target_y: float, elbow_up: bool = True) -> Tuple[bool, Tuple[float, float]]:
        """
        Perform inverse kinematics to reach a target position.
        
        Args:
            target_x, target_y: Target position in meters
            elbow_up: Elbow configuration (True=up, False=down)
            
        Returns:
            (success, (shoulder_angle, elbow_angle))
        """
        if self.arm_2d is None:
            print("❌ Arm 2D solver not initialized")
            return False, (0, 0)
        
        success, angles = self.arm_2d.inverse_kinematics(target_x, target_y, elbow_up)
        
        if success:
            shoulder_angle, elbow_angle = angles
            self.set_arm_pose(shoulder_angle, elbow_angle)
            return True, angles
        else:
            # Try alternative elbow configuration
            success_alt, angles_alt = self.arm_2d.inverse_kinematics(target_x, target_y, not elbow_up)
            if success_alt:
                print(f"💡 Using alternative elbow configuration for ({target_x:.2f}, {target_y:.2f})")
                shoulder_angle, elbow_angle = angles_alt
                self.set_arm_pose(shoulder_angle, elbow_angle)
                return True, angles_alt
            else:
                print(f"⚠️ Target ({target_x:.2f}, {target_y:.2f}) is unreachable")
                print(f"   Distance: {np.sqrt(target_x**2 + target_y**2):.3f}m")
                print(f"   Max reach: {self.arm_2d.max_reach:.3f}m")
                return False, (0, 0)
    
    def execute_reaching_movement(self, start_pos: Tuple[float, float], 
                                end_pos: Tuple[float, float],
                                duration: float = 2.0,
                                trajectory_type: str = 'minimum_jerk',
                                num_points: int = 50) -> Dict:
        """
        Execute a complete reaching movement with smooth trajectory.
        
        Args:
            start_pos: Starting (x, y) position
            end_pos: Target (x, y) position  
            duration: Movement duration in seconds
            trajectory_type: 'minimum_jerk', 'bell_shaped', 'polynomial', 'linear'
            num_points: Number of trajectory points
            
        Returns:
            Dictionary with trajectory data and results
        """
        print(f"\n🎯 Executing reaching movement:")
        print(f"   Start: ({start_pos[0]:.2f}, {start_pos[1]:.2f})")
        print(f"   End: ({end_pos[0]:.2f}, {end_pos[1]:.2f})")
        print(f"   Duration: {duration:.1f}s, Type: {trajectory_type}")
        
        # Generate smooth trajectory
        trajectory = self.trajectory_generator.generate_trajectory(
            start_pos, end_pos, duration, num_points, trajectory_type
        )
        
        # Execute movement by solving IK for each trajectory point
        time_points = []
        positions = []
        joint_angles = []
        success_flags = []
        
        for i, point in enumerate(trajectory):
            t = i * duration / (num_points - 1)
            x, y = point[0], point[1]
            
            success, angles = self.reach_to_target(x, y)
            
            time_points.append(t)
            positions.append((x, y))
            joint_angles.append(angles)
            success_flags.append(success)
        
        # Compile results
        results = {
            'trajectory_type': trajectory_type,
            'duration': duration,
            'num_points': num_points,
            'time_points': time_points,
            'positions': positions,
            'joint_angles': joint_angles,
            'success_flags': success_flags,
            'success_rate': sum(success_flags) / len(success_flags)
        }
        
        print(f"✅ Movement completed! Success rate: {results['success_rate']:.1%}")
        
        return results
    
    def create_motion_file(self, movement_results: Dict, filename: str):
        """
        Create an OpenSim .mot file from movement results.
        
        Args:
            movement_results: Results from execute_reaching_movement()
            filename: Output .mot filename
        """
        print(f"\n💾 Creating motion file: {filename}")
        
        time_points = movement_results['time_points']
        joint_angles = movement_results['joint_angles']
        
        # Create motion file content
        with open(filename, 'w') as f:
            # Header
            f.write(f"name {filename}\n")
            f.write("datacolumns {}\n".format(len(self.arm_coords) + 1))
            f.write("datarows {}\n".format(len(time_points)))
            f.write("range {} {}\n".format(time_points[0], time_points[-1]))
            f.write("endheader\n")
            
            # Column headers
            headers = ["time"]
            for joint_type in ['shoulder', 'elbow']:
                if joint_type in self.arm_coords:
                    headers.append(self.arm_coords[joint_type])
            f.write("\t".join(headers) + "\n")
            
            # Data rows
            for i, t in enumerate(time_points):
                if i < len(joint_angles) and movement_results['success_flags'][i]:
                    shoulder_angle, elbow_angle = joint_angles[i]
                    data_row = [f"{t:.6f}", f"{shoulder_angle:.6f}", f"{elbow_angle:.6f}"]
                    f.write("\t".join(data_row) + "\n")
        
        print(f"✅ Motion file created: {filename}")
        print(f"   - {len(time_points)} time points")
        print(f"   - Load this in OpenSim GUI to visualize the movement!")


class TrajectoryGenerator:
    """Trajectory generator from Day 3 (included for completeness)."""
    
    @staticmethod
    def minimum_jerk_profile(t_norm):
        """Minimum jerk velocity profile: 10t³ - 15t⁴ + 6t⁵"""
        return 10 * t_norm**3 - 15 * t_norm**4 + 6 * t_norm**5
    
    def generate_trajectory(self, start_pos, end_pos, duration, num_points, trajectory_type='minimum_jerk'):
        """Generate smooth trajectory between start and end positions."""
        
        # Time vector
        t = np.linspace(0, duration, num_points)
        t_norm = t / duration  # Normalized time [0, 1]
        
        # Generate position profile based on type
        if trajectory_type == 'minimum_jerk':
            s = self.minimum_jerk_profile(t_norm)
        elif trajectory_type == 'linear':
            s = t_norm
        elif trajectory_type == 'bell_shaped':
            s = 0.5 * (1 - np.cos(np.pi * t_norm))
        else:
            s = t_norm  # Default to linear
        
        # Interpolate between start and end positions
        start_pos = np.array(start_pos)
        end_pos = np.array(end_pos)
        
        trajectory = []
        for i in range(num_points):
            pos = start_pos + s[i] * (end_pos - start_pos)
            trajectory.append(pos)
        
        return trajectory


class Arm2D:
    """2D arm inverse kinematics solver from Day 2 (enhanced for expanded range)."""
    
    def __init__(self, L1, L2, shoulder_offset=(0, 0)):
        self.L1 = L1  # Upper arm length
        self.L2 = L2  # Forearm length
        self.shoulder_offset = shoulder_offset  # Shoulder position offset
        self.max_reach = L1 + L2
        self.min_reach = abs(L1 - L2)
        
        print(f"🦾 2D Arm initialized:")
        print(f"   - Reachable range: {self.min_reach:.2f}m to {self.max_reach:.2f}m")
        print(f"   - Workspace area: {np.pi * self.max_reach**2:.3f} m²")
    
    def inverse_kinematics(self, x, y, elbow_up=True):
        """
        Solve 2-link arm inverse kinematics with elbow configuration choice.
        
        Args:
            x, y: Target position
            elbow_up: True for elbow-up solution, False for elbow-down
        
        Returns:
            (success, (shoulder_angle, elbow_angle))
        """
        # Adjust target relative to shoulder position
        x_rel = x - self.shoulder_offset[0]
        y_rel = y - self.shoulder_offset[1]
        
        # Distance to target
        r = np.sqrt(x_rel**2 + y_rel**2)
        
        # Check reachability with small tolerance
        tolerance = 0.001  # 1mm tolerance
        if r > (self.max_reach + tolerance) or r < (self.min_reach - tolerance):
            return False, (0, 0)
        
        # Clamp distance to valid range
        r = np.clip(r, self.min_reach, self.max_reach)
        
        # Law of cosines for elbow angle
        cos_elbow = (self.L1**2 + self.L2**2 - r**2) / (2 * self.L1 * self.L2)
        cos_elbow = np.clip(cos_elbow, -1, 1)  # Numerical safety
        
        # Elbow angle (choose configuration)
        if elbow_up:
            elbow_angle = np.pi - np.arccos(cos_elbow)  # Positive elbow angle
        else:
            elbow_angle = np.arccos(cos_elbow) - np.pi   # Negative elbow angle
        
        # Shoulder angle calculation
        alpha = np.arctan2(y_rel, x_rel)  # Angle to target
        beta = np.arccos((self.L1**2 + r**2 - self.L2**2) / (2 * self.L1 * r))
        
        if elbow_up:
            shoulder_angle = alpha - beta
        else:
            shoulder_angle = alpha + beta
        
        return True, (shoulder_angle, elbow_angle)
    
    def get_workspace_boundary(self, num_points=1000):
        """Get the boundary of the reachable workspace."""
        angles = np.linspace(0, 2*np.pi, num_points)
        
        # Outer boundary (maximum reach)
        outer_x = self.max_reach * np.cos(angles) + self.shoulder_offset[0]
        outer_y = self.max_reach * np.sin(angles) + self.shoulder_offset[1]
        
        # Inner boundary (minimum reach) - only if min_reach > 0
        if self.min_reach > 0.01:
            inner_x = self.min_reach * np.cos(angles) + self.shoulder_offset[0]
            inner_y = self.min_reach * np.sin(angles) + self.shoulder_offset[1]
            return (outer_x, outer_y), (inner_x, inner_y)
        else:
            return (outer_x, outer_y), None


def load_and_analyze_model(model_path):
    """
    Load OpenSim model and extract all important information (Day 1 function)
    
    Args:
        model_path (str): Path to .osim model file
    
    Returns:
        tuple: (model_info dict, model object, state object)
    """
    try:
        # Load the model
        print(f"Loading model: {model_path}")
        model = osim.Model(model_path)
        
        # Initialize the system (important step!)
        state = model.initSystem()
        
        print(f"✓ Model loaded successfully: {model.getName()}")
        print(f"✓ Model description: {model.getDescription()}")
        print("=" * 50)
        
        # Get model information
        model_info = {
            'name': model.getName(),
            'description': model.getDescription(),
            'coordinates': [],
            'bodies': [],
            'joints': [],
            'muscles': []
        }
        
        # 1. COORDINATES (joint angles/positions)
        print("\n📐 COORDINATES (Joint Angles):")
        coord_set = model.getCoordinateSet()
        print(f"Total coordinates: {coord_set.getSize()}")
        
        for i in range(coord_set.getSize()):
            coord = coord_set.get(i)
            coord_info = {
                'name': coord.getName(),
                'motion_type': coord.getMotionType().name if hasattr(coord.getMotionType(), 'name') else str(coord.getMotionType()),
                'default_value': coord.getDefaultValue(),
                'range_min': coord.getRangeMin(),
                'range_max': coord.getRangeMax(),
                'locked': coord.getDefaultLocked()
            }
            model_info['coordinates'].append(coord_info)
            
            print(f"  {i+1}. {coord.getName()}")
            print(f"     Type: {coord_info['motion_type']}")
            print(f"     Range: [{coord.getRangeMin():.3f}, {coord.getRangeMax():.3f}]")
            print(f"     Default: {coord.getDefaultValue():.3f}")
            print(f"     Locked: {coord.getDefaultLocked()}")
        
        # 2. BODIES (rigid body segments)
        print(f"\n🦴 BODIES (Rigid Segments):")
        body_set = model.getBodySet()
        print(f"Total bodies: {body_set.getSize()}")
        
        for i in range(body_set.getSize()):
            body = body_set.get(i)
            body_info = {
                'name': body.getName(),
                'mass': body.getMass()
            }
            model_info['bodies'].append(body_info)
            
            print(f"  {i+1}. {body.getName()}")
            print(f"     Mass: {body.getMass():.3f} kg")
        
        # 3. JOINTS (connections between bodies)
        print(f"\n🔗 JOINTS (Connections):")
        joint_set = model.getJointSet()
        print(f"Total joints: {joint_set.getSize()}")
        
        for i in range(joint_set.getSize()):
            joint = joint_set.get(i)
            joint_info = {
                'name': joint.getName(),
                'type': joint.getConcreteClassName(),
                'parent_body': joint.getParentFrame().getName(),
                'child_body': joint.getChildFrame().getName()
            }
            model_info['joints'].append(joint_info)
            
            print(f"  {i+1}. {joint.getName()}")
            print(f"     Type: {joint.getConcreteClassName()}")
            print(f"     Connects: {joint.getParentFrame().getName()} → {joint.getChildFrame().getName()}")
        
        # 4. MUSCLES (if any)
        print(f"\n💪 MUSCLES:")
        muscle_set = model.getMuscles()
        print(f"Total muscles: {muscle_set.getSize()}")
        
        if muscle_set.getSize() > 0:
            for i in range(min(5, muscle_set.getSize())):  # Show first 5 only
                muscle = muscle_set.get(i)
                muscle_info = {
                    'name': muscle.getName(),
                    'max_force': muscle.getMaxIsometricForce()
                }
                model_info['muscles'].append(muscle_info)
                
                print(f"  {i+1}. {muscle.getName()}")
                print(f"     Max Force: {muscle.getMaxIsometricForce():.1f} N")
            
            if muscle_set.getSize() > 5:
                print(f"  ... and {muscle_set.getSize() - 5} more muscles")
        else:
            print("No muscles in this model")
        
        return model_info, model, state
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None


# Demo and testing functions
def demo_reaching_system():
    """
    Demonstration of the complete reaching system.
    Replace 'your_model.osim' with your actual model path.
    """
    
    print("=" * 60)
    print("🚀 DAY 4: OpenSim Integration Demo")
    print("=" * 60)
    
    # NOTE: Replace this with your actual .osim model path
    model_path = "W:/Models/Models/Arm26/arm26.osim" # Example: OpenSim's standard arm model
    
    try:
        # Initialize the reaching system
        reaching_system = OpenSimReachingSystem(model_path)
        
        # Define a reaching movement
        start_position = (20, 30)   # Start position (20cm right, 30cm up)
        target_position = (40, 10)  # Target position (40cm right, 10cm up) 
        
        # Execute smooth reaching movement
        results = reaching_system.execute_reaching_movement(
            start_position, 
            target_position,
            duration=3.0,
            trajectory_type='minimum_jerk',
            num_points=1000
        )
        
        # Create OpenSim motion file
        motion_filename = f"reaching_movement_{int(time.time())}.mot"
        reaching_system.create_motion_file(results, motion_filename)
        
        # Print movement summary
        print(f"\n📊 Movement Summary:")
        print(f"   - Trajectory type: {results['trajectory_type']}")
        print(f"   - Duration: {results['duration']:.1f}s")  
        print(f"   - Success rate: {results['success_rate']:.1%}")
        print(f"   - Motion file: {motion_filename}")
        
        print(f"\n🎉 SUCCESS! Your OpenSim model can now perform smooth reaching!")
        print(f"📁 Load '{motion_filename}' in OpenSim GUI to see the animation!")
        
    except FileNotFoundError:
        print(f"❌ Model file '{model_path}' not found.")
        print("📝 Please update the model_path variable with your .osim file path.")
        print("💡 You can download sample models from OpenSim.org")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Make sure OpenSim is properly installed: pip install opensim")


def test_expanded_workspace():
    """Test the expanded workspace with various reaching movements."""
    
    model_path = "W:/Models/Models/Arm26/arm26.osim"  # Update this path
    
    try:
        reaching_system = OpenSimReachingSystem(model_path)
        
        print(f"\n🌟 EXPANDED WORKSPACE TESTING")
        print(f"=" * 50)
        
        # Show workspace info
        if reaching_system.arm_2d:
            print(f"💪 Arm Capabilities:")
            print(f"   - Max reach: {reaching_system.arm_2d.max_reach:.3f}m")
            print(f"   - Min reach: {reaching_system.arm_2d.min_reach:.3f}m") 
            print(f"   - Workspace area: {np.pi * reaching_system.arm_2d.max_reach**2:.3f} m²")
        
        # Test movements across expanded workspace
        test_movements = [
            # Close targets
            ((1, 2), (2, 3), "Close reach"),
            # Medium targets  
            ((2, 1), (4, 4), "Medium reach"),
            # Far targets (using expanded range)
            ((3, 2), (6, 1), "Far reach"),
            # Very far targets
            ((1, 1), (6.5, 0.0), "Maximum reach"),
            # High targets
            ((2, 1), (3, 6), "High reach"),
            # Low targets
            ((3, 5), (5, -1), "Low reach"),
            # Diagonal sweeps
            ((-2, 3), (5, -2), "Diagonal sweep"),
            # Behind-to-front
            ((-3, 2), (4, 3), "Behind-to-front")
        ]
        
        successful_movements = 0
        
        for i, (start, end, description) in enumerate(test_movements):
            print(f"\n--- Test {i+1}: {description} ---")
            print(f"Start: ({start[0]:.2f}, {start[1]:.2f}) → End: ({end[0]:.2f}, {end[1]:.2f})")
            
            # Calculate distances
            start_dist = np.sqrt(start[0]**2 + start[1]**2)
            end_dist = np.sqrt(end[0]**2 + end[1]**2)
            print(f"Distances: Start={start_dist:.3f}m, End={end_dist:.3f}m")
            
            try:
                results = reaching_system.execute_reaching_movement(
                    start, end, 
                    duration=2.5,  # Slightly longer for far reaches
                    trajectory_type='minimum_jerk',
                    num_points=600  # More points for smoother long movements
                )
                
                if results['success_rate'] > 0.8:  # 80% success threshold
                    filename = f"expanded_movement_{i+1}_{description.replace(' ', '_').lower()}.mot"
                    reaching_system.create_motion_file(results, filename)
                    print(f"✅ SUCCESS: {results['success_rate']:.1%} - Motion file: {filename}")
                    successful_movements += 1
                else:
                    print(f"⚠️ PARTIAL: {results['success_rate']:.1%} success rate")
                    
            except Exception as e:
                print(f"❌ FAILED: {e}")
        
        # Summary
        print(f"\n🎊 WORKSPACE TEST SUMMARY")
        print(f"=" * 50)
        print(f"✅ Successful movements: {successful_movements}/{len(test_movements)}")
        print(f"💪 Success rate: {successful_movements/len(test_movements):.1%}")
        
        if successful_movements >= len(test_movements) * 0.75:
            print(f"🌟 EXCELLENT! Expanded workspace is working well!")
        elif successful_movements >= len(test_movements) * 0.5:
            print(f"✅ GOOD! Most movements successful in expanded range")
        else:
            print(f"⚠️ Some movements failed - may need further expansion")
            
        return reaching_system
        
    except Exception as e:
        print(f"❌ Workspace test failed: {e}")
        return None


if __name__ == "__main__":
    print("🎯 Day 4: OpenSim Integration")
    print("Connecting smooth trajectories to OpenSim models!")
    print("-" * 50)
    
    # Run the demo
    demo_reaching_system()
    
    # Uncomment to run additional tests:
    # test_multiple_movements()